In [3]:
# Cell 1: 导入库
import requests
import pandas as pd
import time
import random
from bs4 import BeautifulSoup
import datetime
import re
import os

In [4]:


# Cell 2: 爬虫类
class BossSpiderAPI:
    def __init__(self):
        """金融行业职位爬虫 - 支持断点续爬 (优化版)"""
        self.base_url = "https://www.zhipin.com/wapi/zpgeek/search/joblist.json"
        
        # ！！！每次触发风控后，更新这里的 Cookie！！！
        # 优化：修正 User-Agent 为主流稳定版本，添加 Sec 头模拟真实浏览器
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
            "Referer": "https://www.zhipin.com/web/geek/job",
            "Accept": "application/json, text/plain, */*",
            # "Content-Type": "application/x-www-form-urlencoded", # requests会自动处理，无需手动设置
            "Sec-Ch-Ua": '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
            "Sec-Ch-Ua-Mobile": "?0",
            "Sec-Ch-Ua-Platform": '"Windows"',
            "Sec-Fetch-Dest": "empty",
            "Sec-Fetch-Mode": "cors",
            "Sec-Fetch-Site": "same-origin",
            "Cookie": "ab_guid=78d319c7-13e6-4a47-baf0-f120a7373d6f; lastCity=100010000; wt2=Dkk1F98VQmqMn0UgGwTzC7o7OoHUbYqVCcZHsIQC0DajHRfhzMgYzgV38ljcbzyds0dOdNNbVyWhaO38G_gxErg~~; wbg=0; zp_at=LZazoHzQtE86zsLEFo3PmN_T2jRcXSEvG-dxQ9Lh-JI~; bst=V2R9MnF-L5219pVtRuyRgYLSy27DrTxS0~|R9MnF-L5219pVtRuyRgYLSy27DrRzSk~; __zp_stoken__=9a9agPErDpMODwprDgz8pO0U8NkE4QTxKQz5HPEpZLz0%2FQThLRhkkw4PChmUnYsOIw7TDhz03QzhFPEpBOD5BJ0NEPT9KODk9xL7DgUA5wrBQxIU9KCvDlcOBwoVrJWTDkQwrCcKTw4EMaSzEgcOKDMKnwr0oMRIPWQwIXWQWDA9nDw8PZBYUXhIPZl9fYmpcCBsMZAgTHRw8w63Cv8Kow4fDtMK8wq7DhcOyw4XCrcOLPT9BODM5xL7CsDBLSj1EQ0s4xL7FgMS5xYLFg8S%2BxYDDicOaw7zDqsWAxLnFgsK8xL7FgMS5xILDnMS%2BxYDEucWCw73CvzNDwprCvcO5w4rDsnfDrMSdwqDCocKYT8OyXsKxcMOGwr%2FCsknDgsOBwp7CtMKWw4rCuk%2FCrMKvUsKuwpldwr5mSFt6WFFUa2lnTmFaSXZZGxFeeV4QQwkrSMOP" 
            # 注意：请务必填入最新的 Cookie，否则必报 37 错误
        }
        
        self.finance_pattern = re.compile(
            r"(金融|银行|券商|证券|基金|保险|期货|信托|投行|资管|理财|财富|"
            r"小贷|消金|保理|租赁|征信|清算|支付|互金|金融科技|FinTech)",
            re.IGNORECASE
        )
        
        self.progress_file = "job_data_IN_PROGRESS.xlsx"
        self.state_file = "crawl_state.txt"
        self.detail_state_file = "detail_state.txt"
        
        self.data_list = []
        self.seen_ids = set()

    def is_finance_related(self, job):
        """判断是否为金融相关职位"""
        text = " ".join([
            str(job.get("brandIndustry", "") or ""),
            str(job.get("brandName", "") or ""),
            str(job.get("jobName", "") or "")
        ])
        return bool(self.finance_pattern.search(text))

    def load_crawl_state(self, keywords):
        """加载爬取状态"""
        if not os.path.exists(self.state_file):
            return 0, 1
        
        try:
            with open(self.state_file, 'r', encoding='utf-8') as f:
                lines = f.readlines()
                if len(lines) < 2: return 0, 1
                keyword = lines[0].strip()
                page = int(lines[1].strip())
                
                if keyword in keywords:
                    idx = keywords.index(keyword)
                    print(f"📍 恢复列表爬取进度：关键词 [{keyword}] 第 {page} 页")
                    return idx, page
                else:
                    return 0, 1
        except Exception as e:
            print(f"⚠️ 读取状态文件失败: {e}，从头开始")
            return 0, 1

    def save_crawl_state(self, keyword, page):
        """保存爬取状态"""
        try:
            with open(self.state_file, 'w', encoding='utf-8') as f:
                f.write(f"{keyword}\n{page}")
        except Exception:
            pass

    def load_detail_state(self):
        """加载详情抓取进度"""
        if not os.path.exists(self.detail_state_file):
            return 0
        try:
            with open(self.detail_state_file, 'r', encoding='utf-8') as f:
                index = int(f.read().strip())
                print(f"📍 恢复详情抓取进度：从第 {index + 1} 条开始")
                return index
        except Exception:
            return 0

    def save_detail_state(self, index):
        """保存详情抓取进度"""
        try:
            with open(self.detail_state_file, 'w', encoding='utf-8') as f:
                f.write(str(index))
        except Exception:
            pass

    def fetch_data(self, query="金融", start_page=1, max_pages=5):
        """抓取职位列表数据"""
        for page in range(start_page, max_pages + 1):
            print(f"\n🔍 抓取关键词 [{query}] 列表第 {page} 页...")
            
            # 随机长暂停，模拟人类翻页思考
            if page > 1 and page % 5 == 0:
                 time.sleep(random.uniform(5.0, 8.0))

            params = {
                "scene": "1",
                "query": query,
                "city": "100010000",
                "page": page,
                "pageSize": 30
            }
            
            try:
                resp = requests.post(
                    self.base_url, 
                    headers=self.headers, 
                    data=params, # requests 会自动添加 form-urlencoded 头
                    timeout=15
                )
                resp.raise_for_status()
            except requests.RequestException as e:
                print(f"❌ 请求失败：{e}")
                continue

            try:
                result = resp.json()
                code = result.get("code")
                msg = result.get("message") or result.get("msg")
                
                if code == 37:
                    print(f"⚠️ 触发风控 (code: 37)！")
                    print(f"📍 当前进度：关键词 [{query}] 第 {page} 页")
                    self.save_crawl_state(query, page)
                    self.save_progress()
                    return False
                elif code not in (0, "0", None):
                    print(f"⚠️ 接口异常 code: {code}, message: {msg}")
                    break
                    
            except Exception as e:
                print(f"❌ 解析响应失败: {e}")
                continue

            job_list = result.get("zpData", {}).get("jobList", [])
            
            if not job_list:
                print("⚠️ 本页无数据，进入下一个关键词")
                break
            
            print(f"✅ 获取到 {len(job_list)} 个职位")
            
            finance_count = 0
            for job in job_list:
                job_id = job.get("encryptJobId")
                
                if not job_id or job_id in self.seen_ids:
                    continue
                
                if not self.is_finance_related(job):
                    continue
                
                finance_count += 1
                
                item = {
                    "职位": job.get("jobName"),
                    "公司": job.get("brandName"),
                    "薪资": job.get("salaryDesc"),
                    "地区": job.get("cityName"),
                    "经验": job.get("jobExperience"),
                    "学历": job.get("jobDegree"),
                    "公司规模": job.get("brandScaleName"),
                    "行业": job.get("brandIndustry"),
                    "福利标签": ",".join(job.get("welfareList", []) or []),
                    "技能标签": ",".join(job.get("skills", []) or []),
                    "职位描述": "",
                    "job_id": job_id
                }
                
                self.data_list.append(item)
                self.seen_ids.add(job_id)
            
            print(f"📊 本页符合条件: {finance_count} 个 | 累计: {len(self.data_list)} 个")
            
            self.save_crawl_state(query, page + 1)
            self.save_progress()
            
            # 稍微延长等待时间以降低风控概率
            time.sleep(random.uniform(3.0, 6.0))
        
        return True

    def get_job_detail(self, job_id):
        """获取职位详情描述 - 增强版"""
        url = f"https://www.zhipin.com/job_detail/{job_id}.html"
        
        # 复制主 Header 并针对详情页做微调
        detail_headers = self.headers.copy()
        detail_headers.update({
            "Referer": "https://www.zhipin.com/web/geek/job", # 模拟来源
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8",
            "Sec-Fetch-Dest": "document",
            "Sec-Fetch-Mode": "navigate",
            "Sec-Fetch-User": "?1",
            "Upgrade-Insecure-Requests": "1"
        })
        
        try:
            resp = requests.get(url, headers=detail_headers, timeout=15)
            
            if resp.status_code == 404:
                return "职位已关闭或不存在"
            if resp.status_code == 403:
                print(f"  ⚠️ 403 禁止访问")
                return None
            if resp.status_code != 200:
                print(f"  ⚠️ 状态码: {resp.status_code}")
                return None
            
            if 'security-check' in resp.url or 'verify' in resp.url:
                print(f"  ⚠️ 触发安全验证跳转")
                return None
            
            soup = BeautifulSoup(resp.text, 'html.parser')
            
            selectors = [
                '.job-sec-text', '.job-detail-section .text', '.job-sec .text',
                '.job-description', '.job-detail .text', '[class*="job-sec"] .text'
            ]
            
            for selector in selectors:
                desc_tag = soup.select_one(selector)
                if desc_tag and desc_tag.text.strip():
                    return desc_tag.text.strip()
            
            # 如果没找到，检查是否页面加载了但结构变了
            if "职位描述" in resp.text:
                print(f"  ⚠️ 页面含关键词但未匹配到选择器")
            return ""
            
        except requests.exceptions.RequestException as e:
            print(f"  ⚠️ 详情页请求异常: {e}")
            return None

    def load_progress(self):
        """从进度文件中加载数据"""
        print(f"\n🔄 正在从 {self.progress_file} 加载进度...")
        try:
            df = pd.read_excel(self.progress_file)
            df['职位描述'] = df['职位描述'].fillna('')
            df['job_id'] = df['job_id'].fillna('')
            
            self.data_list = df.to_dict('records')
            self.seen_ids = {item['job_id'] for item in self.data_list if item.get('job_id')}
            
            print(f"✅ 加载成功，已恢复 {len(self.data_list)} 条记录。")
            
        except FileNotFoundError:
            print("ℹ️ 未找到进度文件，将创建新文件。")
        except Exception as e:
            print(f"❌ 加载进度文件失败: {e}")

    def save_progress(self):
        """将当前数据保存到进度文件"""
        if not self.data_list:
            return
        try:
            df = pd.DataFrame(self.data_list)
            df.to_excel(self.progress_file, index=False, na_rep='')
        except Exception as e:
            print(f"  ❌ 保存进度失败: {e}")

    def fetch_all_details_resumable(self, start_index=0):
        """可恢复地抓取所有详情 - 逻辑优化版"""
        print(f"\n{'='*60}")
        print(f"🚀 (阶段2) 开始抓取职位详细描述...")
        if start_index > 0:
            print(f"📍 跳过前 {start_index} 条，直接从第 {start_index + 1} 条开始")
        print(f"{'='*60}")
        
        fetched_count = 0
        failed_count = 0
        consecutive_fail = 0
        
        # 优化：直接从 start_index 开始切片遍历，避免无效循环
        total_items = len(self.data_list)
        
        for i in range(start_index, total_items):
            item = self.data_list[i]
            
            # 检查是否已有描述（数据恢复的情况）
            desc_str = str(item.get("职位描述", "")).strip()
            if desc_str and desc_str.lower() != 'nan' and len(desc_str) > 10:
                continue
            
            print(f"  ({i+1}/{total_items}) [抓取] {item['职位']} - {item['公司']}")
            
            job_id = item.get("job_id")
            if not job_id:
                continue
            
            new_desc = self.get_job_detail(job_id)
            
            if new_desc is None: # 失败
                consecutive_fail += 1
                failed_count += 1
                print(f"  ❌ 抓取失败 (连续: {consecutive_fail})")
                self.save_detail_state(i) # 记录当前失败位置，下次从这里开始
                
                if consecutive_fail >= 5:
                    print(f"\n  🛑 连续失败触发风控保护，停止运行。")
                    self.save_progress()
                    return i
                
                time.sleep(random.uniform(10.0, 20.0)) # 失败后长等待
                
            elif new_desc == "": # 空内容
                consecutive_fail = 0
                item["职位描述"] = "[未获取到描述]"
                print(f"  ⚠️ 未找到描述内容")
                time.sleep(random.uniform(2.0, 4.0))
                
            else: # 成功
                consecutive_fail = 0
                item["职位描述"] = new_desc
                fetched_count += 1
                print(f"  ✅ 获取成功")
                
                # 成功后保存状态（每5条保存一次，减少IO）
                if fetched_count % 5 == 0:
                    self.save_progress()
                    self.save_detail_state(i + 1)
                
                time.sleep(random.uniform(3.0, 6.0)) # 正常间隔
        
        # 循环结束后的保存
        self.save_progress()
        self.save_detail_state(total_items)
        
        print("\n✅ (阶段2) 详情抓取完毕。")
        if os.path.exists(self.detail_state_file):
            os.remove(self.detail_state_file)
        
        return -1

    def save_final_excel(self):
        """保存为最终的 Excel 文件"""
        if not self.data_list:
            return
        
        final_data = [item.copy() for item in self.data_list]
        for item in final_data:
            item.pop('job_id', None) # 移除 job_id
                
        df = pd.DataFrame(final_data)
        filename = f"金融行业岗位_完整版_{datetime.datetime.now().strftime('%Y%m%d_%H%M')}.xlsx"
        df.to_excel(filename, index=False)
        print(f"\n🎉 全部完成！已保存：{filename} (共 {len(df)} 条)")

    def run(self, keywords, max_pages_per_kw=10, skip_list=False):
        """主运行逻辑"""
        # 加载数据
        if os.path.exists(self.progress_file):
            self.load_progress()
        
        # 阶段1：抓取列表
        if not skip_list:
            start_kw_idx, start_page = self.load_crawl_state(keywords)
            
            print(f"\n🚀 (阶段1) 职位列表抓取")
            for kw_idx in range(start_kw_idx, len(keywords)):
                keyword = keywords[kw_idx]
                page = start_page if kw_idx == start_kw_idx else 1
                
                print(f"\n>>> 正在抓取: {keyword} ({kw_idx+1}/{len(keywords)})")
                success = self.fetch_data(query=keyword, start_page=page, max_pages=max_pages_per_kw)
                
                if not success:
                    print("\n⚠️ 程序暂停：请更新 Cookie 后重新运行")
                    return
                
                time.sleep(random.uniform(2.0, 4.0))
            
            if os.path.exists(self.state_file):
                os.remove(self.state_file)
        
        # 阶段2：抓取详情
        start_detail_index = self.load_detail_state()
        if self.data_list:
            result = self.fetch_all_details_resumable(start_index=start_detail_index)
            if result >= 0:
                print("\n⚠️ 详情抓取暂停：请更新 Cookie 后重新运行")
                return
        
        self.save_final_excel()



In [5]:
# Cell 3: 运行爬虫
if __name__ == "__main__":
    spider = BossSpiderAPI()
    
    # 优化：去除了重复的关键词
    raw_keywords = ["数据分析", "基金经理", "产品", "运营", "证券研究所", "投行", "金融", "银行", "保险"]
    # 保持顺序去重
    unique_keywords = list(dict.fromkeys(raw_keywords))
    
    # 正常运行
    spider.run(keywords=unique_keywords, max_pages_per_kw=30)


🔄 正在从 job_data_IN_PROGRESS.xlsx 加载进度...
✅ 加载成功，已恢复 1749 条记录。
📍 恢复列表爬取进度：关键词 [数据分析] 第 4 页

🚀 (阶段1) 职位列表抓取

>>> 正在抓取: 数据分析 (1/9)

🔍 抓取关键词 [数据分析] 列表第 4 页...
✅ 获取到 30 个职位
📊 本页符合条件: 0 个 | 累计: 1749 个

🔍 抓取关键词 [数据分析] 列表第 5 页...
✅ 获取到 30 个职位
📊 本页符合条件: 0 个 | 累计: 1749 个

🔍 抓取关键词 [数据分析] 列表第 6 页...
✅ 获取到 30 个职位
📊 本页符合条件: 0 个 | 累计: 1749 个

🔍 抓取关键词 [数据分析] 列表第 7 页...
✅ 获取到 30 个职位
📊 本页符合条件: 0 个 | 累计: 1749 个

🔍 抓取关键词 [数据分析] 列表第 8 页...
⚠️ 触发风控 (code: 37)！
📍 当前进度：关键词 [数据分析] 第 8 页

⚠️ 程序暂停：请更新 Cookie 后重新运行
